In [17]:
# !pip install sqlalchemy
# !pip install psycopg2-binary
# !pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [1]:
import os

from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display
from dotenv import load_dotenv


In [2]:
load_dotenv()  # take environment variables from .env.
host="localhost"
database=os.getenv("POSTGRES_DB")
user=os.getenv("POSTGRES_USER")
password=os.getenv("POSTGRES_PASSWORD")
port=os.getenv("POSTGRES_PORT")


In [4]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{hostname}:{port}/{database_name}'.format(hostname=host, user=user, password=password, database_name=database, port=5432), echo=True)

In [5]:
inspector = inspect(engine)
inspector.get_table_names()


2024-06-13 14:53:20,799 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-06-13 14:53:20,800 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-13 14:53:20,803 INFO sqlalchemy.engine.Engine select current_schema()
2024-06-13 14:53:20,804 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-13 14:53:20,806 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-06-13 14:53:20,806 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-13 14:53:20,812 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-13 14:53:20,813 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s]) AND pg_catalog.pg_class.relpersistence != %(relpersistence_1)s AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2024-06-13 14:53:20,

[]